In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('../variants.sqlite')
conn.executescript('''\
PRAGMA cache_size=-4192000;
PRAGMA temp_store=MEMORY;
PRAGMA journal_mode=OFF;
''')

In [ ]:
df_variant_class = pd.merge(
    pd.read_sql('''\
SELECT variant_classification, count(*) AS 'mc3_count'
FROM mc3_selected
GROUP BY variant_classification
ORDER BY mc3_count DESC
''', conn),
    pd.read_sql('''\
SELECT variant_classification, count(*) AS 'gdc_count'
FROM gdc_grouped_callers
GROUP BY variant_classification
ORDER BY gdc_count DESC
''', conn),
    how='outer',
    on='variant_classification'
)

In [4]:
df_variant_class.sum()

variant_classification    Missense_MutationSilent3'UTRFrame_Shift_DelNon...
mc3_count                                                            445856
gdc_count                                                            515859
dtype: object

In [5]:
df_per_tx_sample_variant_type = pd.read_sql('''\
WITH gdc_by_tx AS (
    SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, count(*) AS 'gdc_count'
    FROM gdc_grouped_callers
    GROUP BY hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode
), mc3_by_tx AS (
    SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, count(*) AS 'mc3_count'
    FROM mc3_selected
    GROUP BY hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode
)
SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, gdc_count, mc3_count FROM gdc_by_tx
LEFT JOIN mc3_by_tx 
USING (hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode)
UNION ALL
SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, gdc_count, mc3_count FROM mc3_by_tx
LEFT JOIN gdc_by_tx
USING (hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode)
WHERE gdc_by_tx.gdc_count IS NULL
''', conn)

In [6]:
df_per_tx_sample_variant_type = (
    df_per_tx_sample_variant_type
    .fillna(0)
    .assign(
        gdc_count=lambda df: df.gdc_count.astype(int),
        mc3_count=lambda df: df.mc3_count.astype(int))
    .assign(
        diff=lambda df: df.gdc_count-df.mc3_count
    )
    .sort_values(['diff'], ascending=False)
)

In [7]:
df_per_tx_sample_variant_type.head()

,hugo_symbol,transcript_id,variant_classification,tumor_sample_barcode,gdc_count,mc3_count,diff
438110,TTN,ENST00000591111,Missense_Mutation,TCGA-CA-6717-01A-11D-1835-10,29,0,29
437991,TTN,ENST00000591111,Missense_Mutation,TCGA-AA-A010-01A-01D-A17O-10,29,0,29
438021,TTN,ENST00000591111,Missense_Mutation,TCGA-AN-A046-01A-21W-A050-09,28,0,28
437990,TTN,ENST00000591111,Missense_Mutation,TCGA-AA-A00N-01A-02D-A17O-10,28,0,28
438059,TTN,ENST00000591111,Missense_Mutation,TCGA-AZ-4315-01A-01D-1408-10,23,0,23


In [8]:
df_per_tx_sample_variant_type[
    (df_per_tx_sample_variant_type['hugo_symbol'] == 'TP53') &
    (df_per_tx_sample_variant_type['tumor_sample_barcode'] == 'TCGA-13-0755-01A-01W-0371-08')
]

,hugo_symbol,transcript_id,variant_classification,tumor_sample_barcode,gdc_count,mc3_count,diff
427205,TP53,ENST00000269305,Splice_Region,TCGA-13-0755-01A-01W-0371-08,2,0,2
592756,TP53,ENST00000269305,Silent,TCGA-13-0755-01A-01W-0371-08,0,1,-1


In [9]:
df_per_tx_sample = (
    df_per_tx_sample_variant_type
    .groupby(['hugo_symbol', 'transcript_id', 'tumor_sample_barcode'])
    .sum()
    .sort_values(['diff'], ascending=False)
    .reset_index()
)

In [10]:
df_per_tx_sample.head()

,hugo_symbol,transcript_id,tumor_sample_barcode,gdc_count,mc3_count,diff
0,TTN,ENST00000591111,TCGA-CA-6717-01A-11D-1835-10,68,0,68
1,TTN,ENST00000591111,TCGA-AA-A010-01A-01D-A17O-10,48,0,48
2,TTN,ENST00000591111,TCGA-AN-A046-01A-21W-A050-09,45,0,45
3,TTN,ENST00000591111,TCGA-AZ-4315-01A-01D-1408-10,44,0,44
4,TTN,ENST00000591111,TCGA-AA-A00N-01A-02D-A17O-10,40,0,40


In [11]:
df_per_tx = (
    df_per_tx_sample.groupby(['hugo_symbol', 'transcript_id'])
    .sum()
    .sort_values(['diff'], ascending=False)
    .reset_index()
)

In [12]:
df_per_tx.head(20)

,hugo_symbol,transcript_id,gdc_count,mc3_count,diff
0,TTN,ENST00000591111,1901,0,1901
1,APC,ENST00000257430,616,0,616
2,DST,ENST00000312431,379,0,379
3,OBSCN,ENST00000422127,378,0,378
4,MUC4,ENST00000463781,479,145,334
5,NEB,ENST00000172853,309,0,309
6,TP53,ENST00000269305,1291,992,299
7,FAT3,ENST00000525166,291,0,291
8,MACF1,ENST00000372915,274,0,274
9,CSMD1,ENST00000520002,265,0,265


In [13]:
# Export data to CSV
df_per_tx.to_csv('../variant_count_diff.per_tx.csv', index=False)
df_per_tx_sample.to_csv('../variant_count_diff.per_tx_sample.csv', index=False)
df_per_tx_sample_variant_type.to_csv('../variant_count_diff.per_tx_sample_type.csv', index=False)

In [19]:
df_per_tx[df_per_tx['hugo_symbol'].str.startswith('U2AF')]

,hugo_symbol,transcript_id,gdc_count,mc3_count,diff
3130,U2AF1L4,ENST00000412391,17,0,17
12801,U2AF2,ENST00000308924,16,14,2
22392,U2AF1,ENST00000291552,5,13,-8
22597,U2AF1L4,ENST00000292879,0,9,-9


In [17]:
df_per_tx_sample[df_per_tx_sample['hugo_symbol'] == 'U2AF1']

,hugo_symbol,transcript_id,tumor_sample_barcode,gdc_count,mc3_count,diff
182774,U2AF1,ENST00000291552,TCGA-AN-A0AK-01A-21W-A019-09,1,0,1
182775,U2AF1,ENST00000291552,TCGA-AA-3966-01A-01D-1981-10,1,0,1
182776,U2AF1,ENST00000291552,TCGA-AA-3672-01A-01W-0900-09,1,0,1
182777,U2AF1,ENST00000291552,TCGA-A6-3809-01A-01D-A270-10,1,0,1
232319,U2AF1,ENST00000291552,TCGA-CM-5861-01A-01D-1650-10,1,1,0
475792,U2AF1,ENST00000291552,TCGA-GM-A5PX-01A-12D-A28B-09,0,1,-1
475793,U2AF1,ENST00000291552,TCGA-E2-A1IO-01A-11D-A142-09,0,1,-1
475794,U2AF1,ENST00000291552,TCGA-DM-A1HB-01A-21D-A183-10,0,1,-1
475796,U2AF1,ENST00000291552,TCGA-CM-4744-01A-01D-1408-10,0,1,-1
475797,U2AF1,ENST00000291552,TCGA-AD-6889-01A-11D-1924-10,0,1,-1


In [15]:
pd.set_option('display.max_columns', None)

## TP53

Check if there is any flag for the variants of TP53.

In [ ]:
"group_concat(caller, '|')"

In [23]:
sample = 'TCGA-BH-A0EE-01A-11W-A050-09'
gene = 'TP53'
df_gdc = pd.read_sql(
    '''
    SELECT 
        chromosome, start_position, end_position, 
        variant_classification, variant_type, 
        reference_allele, tumor_seq_allele1, tumor_seq_allele2,
        dbsnp_rs, sequencer,
        "group_concat(t_depth, ',')", "group_concat(t_ref_count, ',')", "group_concat(t_alt_count, ',')",
        "group_concat(n_depth, ',')", "group_concat(caller, '|')"
    FROM gdc_grouped_callers
    WHERE hugo_symbol=? AND tumor_sample_barcode=?
    ''',
    conn,
    params=[gene, sample]
).sort_values(['start_position', 'end_position', "group_concat(caller, '|')"])
df_mc3 = pd.read_sql(
    '''
    SELECT 
        chromosome, start_position, end_position, 
        variant_classification, variant_type, 
        reference_allele, tumor_seq_allele1, tumor_seq_allele2,
        dbsnp_rs,
        t_depth, t_ref_count, t_alt_count, n_depth,
        centers, ncallers
    FROM mc3_selected
    WHERE hugo_symbol=? AND tumor_sample_barcode=?
    ''',
    conn,
    params=[gene, sample]
)

In [24]:
df_gdc

,chromosome,start_position,end_position,variant_classification,variant_type,reference_allele,tumor_seq_allele1,tumor_seq_allele2,dbsnp_rs,sequencer,"group_concat(t_depth, ',')","group_concat(t_ref_count, ',')","group_concat(t_alt_count, ',')","group_concat(n_depth, ',')","group_concat(caller, '|')"
0,chr17,7675079,7675079,Frame_Shift_Del,DEL,T,T,-,novel,Illumina Genome Analyzer II,109,76,32,145,varscan
1,chr17,7675079,7675079,Missense_Mutation,SNP,T,T,G,,Illumina Genome Analyzer II,"99,86","52,58","16,25","145,167",varscan|somaticsniper
2,chr17,7675085,7675085,Missense_Mutation,SNP,C,C,G,,Illumina Genome Analyzer II,"96,115,115","50,66,66","40,45,45","131,153,153",varscan|somaticsniper|muse


In [25]:
df_mc3

,chromosome,start_position,end_position,variant_classification,variant_type,reference_allele,tumor_seq_allele1,tumor_seq_allele2,dbsnp_rs,t_depth,t_ref_count,t_alt_count,n_depth,centers,ncallers
0,17,7675079,7675085,Frame_Shift_Del,DEL,TGGGGGC,TGGGGGC,GGGGGG,novel,67,37,30,69,PINDEL|MUSE*|SOMATICSNIPER*|VARSCANI*|INDELOCA...,7


Dump all TP53 and APC variants

In [33]:
df_dump_mc3 = pd.read_sql(
'''
SELECT *
FROM mc3_selected
WHERE hugo_symbol IN ('TP53', 'APC')
''', conn).sort_values(['hugo_symbol', 'tumor_sample_barcode', 'start_position', 'end_position'])

df_dump_gdc = pd.read_sql(
'''
SELECT *
FROM gdc_grouped_callers
WHERE hugo_symbol IN ('TP53', 'APC')
''', conn).sort_values(['hugo_symbol', 'tumor_sample_barcode', 'start_position', 'end_position'])

In [35]:
df_dump_mc3.to_csv('../mc3_tp53_apc_only.maf')

In [36]:
df_dump_gdc.to_csv('../gdc_tp53_apc_only.maf')